<a href="https://colab.research.google.com/github/sanjayc2/MNIST_w_CNN_NoOverfit_High_Accuracy/blob/main/MNIST_with_CNN_and_Data_Augmentation_and_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [ ]:
keras.__version__
tfds.__version__

'4.0.1'

In [ ]:
(X_train_full, y_train_full), (X_test_full, y_test_full) = keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train_full.shape

(60000, 28, 28)

In [ ]:
X_train, X_valid = X_train_full[:50000]/255, X_train_full[50000:]/255
y_train, y_valid = y_train_full[:50000], y_train_full[50000:]

print(y_train.shape)
print(y_valid.shape)

(50000,)
(10000,)


In [ ]:
# set up TensorBoard
import os
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
  return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

NameError: ignored

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    #zoom_range=[0.5,2],
    validation_split=0.2,
    )

# reshape X_train and X_valid (4-dim tensors required)
X_train_full = X_train_full.reshape(X_train_full.shape[0],X_train_full.shape[1],X_train_full.shape[2],1)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_full)
train_generator = datagen.flow(X_train_full, y_train_full, batch_size=32, shuffle=True, 
                               seed=42, save_to_dir=None, subset='training')

validation_generator = datagen.flow(X_train_full, y_train_full, batch_size=32, shuffle=True, 
                               seed=42, save_to_dir=None, subset='validation')

In [ ]:
# define function for learning rate scheduler callback
import keras.backend as K

class lr_loss_callback(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.learn_rates = []
        self.losses      = []
    def on_epoch_end(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        if not hasattr(self.model.optimizer, "iterations"):
            raise ValueError('Optimizer must have a "iterations" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(K.get_value(self.model.optimizer.lr))
        #if hasattr(self.model.optimizer, "decay"):
        #  decay = self.model.optimizer.decay
        #  iterations = self.model.optimizer.iterations 
        #  lr_updated = lr / (1. + decay * K.cast(iterations, K.dtype(decay)))
        #else:
        lr_updated = lr * self.factor  
        self.learn_rates.append(lr_updated)
        self.losses.append(logs["val_loss"])
        K.set_value(self.model.optimizer.lr, lr_updated)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, lr_updated))
        print("The average validation loss for epoch {} is {:7.4f}".format(epoch, logs["val_loss"]))

In [ ]:
import numpy as np
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
input_ = keras.layers.Input(shape=[28,28,1])
flatten_ = keras.layers.Flatten()(input_)
dropout1 = keras.layers.Dropout(rate=0.2)(flatten_)
hidden1 = keras.layers.Dense(100, activation="relu")(dropout1)
dropout2 = keras.layers.Dropout(rate=0.1)(hidden1)
hidden2 = keras.layers.Dense(50,activation="relu")(dropout2)
dropout3 = keras.layers.Dropout(rate=0.1)(hidden2)
output = keras.layers.Dense(10,activation="softmax")(dropout3)
model = keras.Model(inputs = [input_], outputs = output)
#model.save_weights('model.h5')
optimizer = keras.optimizers.SGD(lr = 0.08)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#model.load_weights('model.h5')
lr_callback = lr_loss_callback(factor=1.00)    # dont change learning rate..
early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    verbose=2,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)
history = model.fit(X_train, y_train, epochs = 100, validation_data=(X_valid, y_valid), callbacks=[lr_callback, early_stopping_callback])
# fits the model on batches with real-time data augmentation:
#history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[lr_callback, early_stopping_callback])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
1548/1563 [============================>.] - ETA: 0s - loss: 0.4455 - accuracy: 0.8618
Epoch 00000: Learning rate is 0.0800.
The average validation loss for epoch 0 is  0.1801
1563/1563 [==============================] - 6s 3ms/step - loss: 0.4440 - accuracy: 0.8623 - val_loss: 0.1801 - val_accuracy: 0.9472
Epoch 2/100
1562/1563 [============================>.] - ETA: 0s - loss: 0.2328 - accuracy: 0.9295
Epoch 00001: Learning rate is 0.0800.
The average validation loss for epoch 1 is  0.1263
1563/1563 [==============================] - 5s 3ms/step - loss: 0.2327 - accuracy: 0.9295 - val_loss: 0.1263 - val_accuracy: 0.9640
Epoch 3/100
1553/1563 [============================>.] - ETA: 0s - loss: 0.1888 - accuracy: 0.9425
Epoch 00002: Learning rate is 0.0800.
The average validation loss for epoch 2 is  0.1100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.1886 - accuracy: 0.9425 - val_loss: 0.1100 - val_accuracy: 0.9663
Epoch 4/100
1552/1563 [==============

In [1]:
X_test = X_test_full
Y_test = Y_test_full
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
#print(type(y_pred.argmax()))
#print(type(y_test))
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, np.argmax(y_pred,axis=1))
print("conf_matrix:", conf_matrix)

pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

NameError: ignored

In [ ]:
plt.plot(lr_loss_update.learn_rates, lr_loss_update.losses)
plt.gca().set_xscale('log')
plt.hlines(min(lr_loss_update.losses), min(lr_loss_update.learn_rates), max(lr_loss_update.learn_rates))
plt.axis([min(lr_loss_update.learn_rates), max(lr_loss_update.learn_rates), 0, lr_loss_update.losses[0]])
plt.grid()
plt.xlabel("Learning rate")
plt.ylabel("Loss")

NameError: ignored

NameError: ignored